In [1]:
import torch.nn as nn
import torch
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [2]:
transform=transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor()
])

In [3]:
data=ImageFolder(root='pc_parts_ready',transform=transform)

In [4]:
dataloader=DataLoader(dataset=data,batch_size=128,shuffle=True)

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=16, kernel_size=4, stride=1, padding=0)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=24, kernel_size=4, stride=1, padding=0)
        self.fc1 = nn.Linear(in_features=53*53*24, out_features=500)
        self.fc2 = nn.Linear(in_features=500, out_features=200)
        self.fc3 = nn.Linear(in_features=200, out_features=80)
        self.fc4 = nn.Linear(in_features=80, out_features=14)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1,53*53*24)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x


In [6]:
data

Dataset ImageFolder
    Number of datapoints: 3043
    Root location: pc_parts_ready
    StandardTransform
Transform: Compose(
               Resize(size=224, interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )

In [7]:
model=CNN()
model

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(4, 4), stride=(1, 1))
  (conv2): Conv2d(16, 24, kernel_size=(4, 4), stride=(1, 1))
  (fc1): Linear(in_features=67416, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=80, bias=True)
  (fc4): Linear(in_features=80, out_features=14, bias=True)
)

In [8]:
optimizer=torch.optim.Adam(model.parameters(), lr=0.001)
criterion=nn.CrossEntropyLoss()

In [9]:
epochs=15  
for epoch in range(epochs):
    for b,(x,y) in enumerate(dataloader):
        y_pred=model(x)
        loss = criterion(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print(f"Loss : {loss} Batch : {b+1} Epoch : {epoch+1}/{epochs}")

Loss : 2.6343867778778076 Batch : 1 Epoch : 1/15
Loss : 2.614001989364624 Batch : 2 Epoch : 1/15
Loss : 2.673722743988037 Batch : 3 Epoch : 1/15
Loss : 2.667572021484375 Batch : 4 Epoch : 1/15
Loss : 2.5885815620422363 Batch : 5 Epoch : 1/15
Loss : 2.6117639541625977 Batch : 6 Epoch : 1/15
Loss : 2.5866594314575195 Batch : 7 Epoch : 1/15
Loss : 2.6487998962402344 Batch : 8 Epoch : 1/15
Loss : 2.5797550678253174 Batch : 9 Epoch : 1/15
Loss : 2.5793960094451904 Batch : 10 Epoch : 1/15
Loss : 2.6324901580810547 Batch : 11 Epoch : 1/15
Loss : 2.61806321144104 Batch : 12 Epoch : 1/15
Loss : 2.610595464706421 Batch : 13 Epoch : 1/15
Loss : 2.6325573921203613 Batch : 14 Epoch : 1/15
Loss : 2.635603189468384 Batch : 15 Epoch : 1/15
Loss : 2.578911304473877 Batch : 16 Epoch : 1/15
Loss : 2.6045172214508057 Batch : 17 Epoch : 1/15
Loss : 2.606374502182007 Batch : 18 Epoch : 1/15
Loss : 2.5894076824188232 Batch : 19 Epoch : 1/15
Loss : 2.562462329864502 Batch : 20 Epoch : 1/15
Loss : 2.5853233337

In [10]:
torch.save(model.state_dict(),"model_weigths.pth")
torch.save(model,'model.pth')

In [11]:
model=torch.load('model.pth')
model.eval()

CNN(
  (conv1): Conv2d(3, 16, kernel_size=(4, 4), stride=(1, 1))
  (conv2): Conv2d(16, 24, kernel_size=(4, 4), stride=(1, 1))
  (fc1): Linear(in_features=67416, out_features=500, bias=True)
  (fc2): Linear(in_features=500, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=80, bias=True)
  (fc4): Linear(in_features=80, out_features=14, bias=True)
)